In [16]:
from dataclasses import dataclass
from typing import List, Tuple


* Interesting other stuff https://stackoverflow.com/questions/68336561/threshold-method-to-detect-dynamically-red-from-orange

In [19]:
@dataclass
class Colour:
    upper: Tuple[int]
    lower: Tuple[int] 

@dataclass
class ColourStore: 
    black: Colour
    white: Colour
    blue: Colour
    red: Colour
    blue: Colour
    red: Colour
    green: Colour
    yellow: Colour
    brown: Colour


colour_store = ColourStore(
    black = Colour(lower=(0, 0, 0), upper=(180, 255, 30)), # https://stackoverflow.com/questions/25398188/black-color-object-detection-hsv-range-in-opencv#:~:text=For%20black%20and%20white%20colors,200%20to%20255%20for%20white.
    white = Colour(lower=(0, 0, 200), upper=(180, 255, 255)), # https://stackoverflow.com/questions/25398188/black-color-object-detection-hsv-range-in-opencv#:~:text=For%20black%20and%20white%20colors,200%20to%20255%20for%20white.
    blue = Colour(lower=(100,150,0), upper=(140,255,255)),
    red = Colour(lower=(155,25,0), upper=(179,255,255)),
    green = Colour(lower=(36, 25, 25), upper=(70, 255,255)), # https://stackoverflow.com/questions/47483951/how-to-define-a-threshold-value-to-detect-only-green-colour-objects-in-an-image
    yellow = Colour(lower=(22, 93, 0), upper=(45, 255, 255)), 
    brown = Colour(lower=(0, 100, 20), upper=(10, 255, 255)), # https://stackoverflow.com/questions/31760302/detect-brown-colour-object-using-opencv 
)

In [26]:
@dataclass
class HeuristicModel:
    """
    Way to create heuristics based on colour range.
    """

    class_list: List[str] = None
    class_name: str = None
    colour: str = None
    lower_bound: List[int] = None
    upper_bound: List[int] = None
    min_width: int = None
    min_height: int = None

    def __post_init__(self): 
        # so that the Colour
        # objects can be accessed by string 
        # in the dictionary
        self.colour_store = colour_store.__dict__

        if self.lower_bound is None:
            colour = self.colour_store[self.colour]
            self.lower_bound = colour.lower
            self.upper_bound = colour.upper

    def predict(self, image_path: str):
        try:
            # Load the image
            image = cv2.imread(image_path)

            # Convert the image to the HSV color space
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Define the lower and upper bounds for white color
            lower_value = np.array(self.lower_bound)
            upper_value = np.array(self.upper_bound)

            # Create a mask for white regions
            mask = cv2.inRange(hsv, lower_value, upper_value)

            # Find contours in the mask
            contours, _ = cv2.findContours(
                mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            )

            # Draw bounding rectangles around white regions
            bounding_boxes = []
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)

                if (w > 20) and (h > 20):
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    if (w > self.min_width) or (h > self.min_height):
                        bounding_boxes.append([x, y, x + w, y + h])

            class_index = self.class_list.index(self.class_name)

            if len(bounding_boxes) != 0:
                return Predictions(
                    scores=[1] * len(bounding_boxes),
                    labels=[class_index] * len(bounding_boxes),
                    boxes=bounding_boxes,
                )
            else:
                return EmptyPredictions

        except Exception as e:
            print(e)
            return EmptyPredictions


In [24]:
colour_store.__dict__

{'black': Colour(upper=(180, 255, 30), lower=(0, 0, 0)),
 'white': Colour(upper=(180, 255, 255), lower=(0, 0, 200)),
 'blue': Colour(upper=(140, 255, 255), lower=(100, 150, 0)),
 'red': Colour(upper=(179, 255, 255), lower=(155, 25, 0)),
 'green': Colour(upper=(70, 255, 255), lower=(36, 25, 25)),
 'yellow': Colour(upper=(45, 255, 255), lower=(22, 93, 0)),
 'brown': Colour(upper=(10, 255, 255), lower=(0, 100, 20))}

In [27]:
heuristic_model = HeuristicModel(
    class_name='wood',
    colour='brown',
)

In [ ]:
import inspect
import torch
from PIL import Image
from torchvision import transforms

import logging

import fiftyone as fo

from vision.tools.pipelines import (
    camera_from_filename,
)

from PIL import Image
from icecream import ic

from vision.tools.predictions import (
    Predictions, 
    EmptyPredictions,
)
from vision.tools.masks import (
    paste_masks_in_image,
)

from typing import List, Tuple



def get_predictions(
        model,
        dataset,
        class_list: list = None,
        output_key = 'predictions',
        device: str ='cpu',
        w: int = 1920, 
        h: int = 1080,
    ):
    
    logging.info(f"""
        Output key is {output_key}, WARNING: getting this wrong leads 
        to horrible downstream bugs with missing labels and out of sync 
        datasets.
        
        For cropping it should be set to detections so that the output
        of the model can be treated the same as GT labels.
        
        For evaluation and thresholding it should be set to predictions, 
        so that the output of the model can be compared to the GT
        labels.
    """)
    assert class_list is not None, 'Need to specify class list'

    with fo.ProgressBar() as pb:
        for sample in pb(dataset):
            try:
                preds = model.predict(sample.filepath)

                # Convert detections to FiftyOne format
                detections = []
                for label, score, box in zip(
                    preds.labels, 
                    preds.scores, 
                    preds.boxes,
                ):
                    # Convert to [top-left-x, top-left-y, width, height]
                    # in relative coordinates in [0, 1] x [0, 1]
                    x1, y1, x2, y2 = box
                    rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]
                    
                    assert int(label) <= (len(class_list) - 1), f'Index {label} is out of class list range'
                    class_name = class_list[label]
                    
                    detection = fo.Detection(
                        label=class_name,
                        bounding_box=rel_box,
                        confidence=score,
                    )

                    detections.append(detection)

                 # Save predictions to dataset
                sample[output_key] = fo.Detections(detections=detections)
                sample.save()
            except Exception as e:
                #raise e
                # need to be more specific with exception handling
                sample[output_key] = fo.Detections(detections=[])
                sample.save()

                print(f'Exception hit == {e}')
                print(f"""
                    WARNING: Possible there are no objects in the frames.
                    Or the model has been trained on more classes than are 
                    included in the provided class_list.
                """)
                logging.info(e)

    return dataset